# Lista 10
Część pierwsza sprawozdania.

## Zadanie 1

In [18]:
#install.packages("kableExtra")
#install.packages("tidyverse")

library(tidyverse)
library(knitr)
library(kableExtra)
library(IRdisplay)

## Zadanie 2

## Zadanie 3

# Lista 11
Część druga sprawozdania.

## Zadanie 1

W zadaniu pierwszym przeprowadzimy test symetrii, quasi-symetrii i quasi-niezależności dla danych dotyczących ocen uzyskanych przez studentów z dwóch kolokwiów. Wszystkie testy zostaną przeprowadzone na poziomie istotności $0.05$.

In [59]:
# HIDE INPUT

count <- c(5, 2, 1, 0, 0, 0,
           6, 3, 2, 2, 0, 0,
           1, 4, 5, 5, 2, 2,
           0, 10, 15, 18, 5, 2,
           1, 2, 5, 3, 2, 2,
           0, 1, 3, 4, 3, 2)

data <- matrix(count, byrow=TRUE, nrow=6)
rownames(data) = c("2", "3", "+3", "4", "+4", "5")
colnames(data) = c("2", "3", "+3", "4", "+4", "5")

data %>% 
    kable("html") %>%
    as.character() %>%
    display_html()

ERROR: Error in table(count, byrow = TRUE, nrow = 6, rownames = c("2", "3", "+3", : all arguments must have the same length


In [58]:
?matrix

### Symetria
Przy testowaniu symetrii, moglibyśmy skorzystać z testu Bowkera, jednak w przypadku naszych danych nie jest to możliwe. Wywołanie tego testu (`mcnemar.test`) zwróciłoby p-wartość `NaN`. Dlatego należy skorzystać z testu ilorazu wiarygodności, zaimplementowanego w funkcji `glm` biblioteki `gnm`.

In [39]:
#install.packages("gnm")
#install.packages("dplyr")
library(gnm)
library(dplyr)

count <- c(5, 2, 1, 0, 0, 0,
           6, 3, 2, 2, 0, 0,
           1, 4, 5, 5, 2, 2,
           0, 10, 15, 18, 5, 2,
           1, 2, 5, 3, 2, 2,
           0, 1, 3, 4, 3, 2)

K1 <- gl(6,6, labels=c("2", "3", "+3", "4", "+4", "5"))
K2 <- gl(6,1, labels=c("2", "3", "+3", "4", "+4", "5"))
data <- data.frame(K1, K2, count)

symmetry <- glm(count ~ Symm(K1, K2), data=data, family=poisson)
symmetry %>% summary()


Call:
glm(formula = count ~ Symm(K1, K2), family = poisson, data = data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8988  -0.5463   0.0000   0.4981   1.4888  

Coefficients:
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)        1.609e+00  4.472e-01   3.599  0.00032 ***
Symm(K1, K2)2:3   -2.231e-01  5.701e-01  -0.391  0.69549    
Symm(K1, K2)2:+3  -1.609e+00  8.367e-01  -1.924  0.05440 .  
Symm(K1, K2)2:4   -1.991e+01  4.043e+03  -0.005  0.99607    
Symm(K1, K2)2:+4  -2.303e+00  1.095e+00  -2.102  0.03556 *  
Symm(K1, K2)2:5   -1.991e+01  4.043e+03  -0.005  0.99607    
Symm(K1, K2)3:3   -5.108e-01  7.303e-01  -0.699  0.48425    
Symm(K1, K2)3:+3  -5.108e-01  6.055e-01  -0.844  0.39889    
Symm(K1, K2)3:4    1.823e-01  5.323e-01   0.343  0.73196    
Symm(K1, K2)3:+4  -1.609e+00  8.367e-01  -1.924  0.05440 .  
Symm(K1, K2)3:5   -2.303e+00  1.095e+00  -2.102  0.03556 *  
Symm(K1, K2)+3:+3  2.561e-15  6.325e-01   0.000  1.00000    
Symm

Policzmy zatem p-wartość.

In [45]:
x = symmetry$deviance
r = 15

p = 1 - pchisq(x, r)
p %>% round(4)

[1] 0.1005

Otrzymujemy p-wartość na poziomie $0.1005 < 0.05$, więc odrzucamy hipotezę o symetrii.

### Quasi-symetria
Ponownie skorzystamy z funkcji `glm`.

In [46]:
quasi_symm <- glm(count ~ K1+K2 + Symm(K1,K2), data=data, family =poisson)
summary(quasi_symm)


Call:
glm(formula = count ~ K1 + K2 + Symm(K1, K2), family = poisson, 
    data = data)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-0.86903  -0.08908   0.00000   0.06047   0.80881  

Coefficients: (5 not defined because of singularities)
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)          1.60944    0.44721   3.599  0.00032 ***
K13                 -1.83252    1.17661  -1.557  0.11936    
K1+3                 0.29184    0.76722   0.380  0.70365    
K14                  1.16686    0.72161   1.617  0.10587    
K1+4                 0.86552    0.77135   1.122  0.26183    
K15                  0.85966    0.63615   1.351  0.17658    
K23                 -2.61466    1.16128  -2.252  0.02435 *  
K2+3                -1.11675    0.74555  -1.498  0.13416    
K24                 -1.28102    0.71818  -1.784  0.07447 .  
K2+4                -1.37441    0.75441  -1.822  0.06848 .  
K25                 -1.77595    0.63615  -2.792  0.00524 ** 

In [51]:
x = quasi_symm$deviance
r = 10
p = 1 - pchisq(x, r)

p %>% round(4)

[1] 0.9589

Otrzymana p-wartość na poziomie $0.9589$ nie daje podstaw do odrzucenia hipotezy o quasi-symetrii.

### Quasi-niezależność
Do testowania quasi-niezależności również możemy wykorzystać funkcję `glm`.

In [53]:
quasi_ind <-  glm(count ~ K1 + K2 + Diag(K1, K2), data=data, family = poisson)
summary(quasi_ind)


Call:
glm(formula = count ~ K1 + K2 + Diag(K1, K2), family = poisson, 
    data = data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.5605  -0.6443  -0.0649   0.2053   3.3845  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -1.3500     0.6876  -1.963 0.049590 *  
K13              1.3547     0.6616   2.048 0.040602 *  
K1+3             1.8306     0.6423   2.850 0.004372 ** 
K14              2.5373     0.6092   4.165 3.12e-05 ***
K1+4             1.5024     0.6425   2.338 0.019370 *  
K15              1.2838     0.6527   1.967 0.049190 *  
K23              0.9628     0.4241   2.270 0.023181 *  
K2+3             1.3646     0.4095   3.333 0.000861 ***
K24              1.0334     0.4545   2.274 0.022994 *  
K2+4             0.3432     0.4768   0.720 0.471570    
K25             -0.1993     0.5417  -0.368 0.712931    
Diag(K1, K2)2    2.9595     0.8202   3.608 0.000308 ***
Diag(K1, K2)3    0.1312     0.7110   0.184 0.853635   

In [55]:
x = quasi_ind$deviance
r = 19
p = 1 - pchisq(x, r)
p %>% round(4)

[1] 0.0096

Test ilorazu wiarygodności zwraca p-wartość $0.0096 < 0.05$, więc odrzucamy hipotezę zerową o quasi-niezależności.

## Zadanie 2

zad 2 
trzeba wskazac ze rozklady brzegowe sa jednorodne

na poprzedniej liscie weryfikowalismy hip ze dane pochodza z symetrii (?)

wynik z testu symetrii jakby jednoznacznie okreslal czy mozemy odrzucic hipoteze o jednorodnosci czy nie

tutaj sytuacja jest bardziej skomplikowana poniewaz symetria nie jest rownowazna jednorodnosci

symetria implikuje jednorodnosc rozkladu (implikacja tylko w 1 strone)

w kontekscie zadania drugiego,
jezeli w podpunkcie a w 1 zadaniu wyjdzie ze nie mozna odrzucic hipotezy, bedziemy wnioskowac ze dane pochodza z modelu symetrii, to automatycznie mowimy ze rozklady brzegowe za jednorodne

jezeli w 1a odrzucamy hipoteze, to nie mozemy automatycznie odrzucic hipotezy

w zadaniu 2 powolujemy sie na wynik z 1a jesli ta nie odrzucimy hipotezy o symetrii

nie bedziemy miec podstaw do odrzucenia wiec mzoemy tak zrobic


drugi test jaki mozna wykonac to test bazujacy na ilorazie wiarygnodnosci

na wykladzie mielismy na kilku slajdach rozpisane ze pod warunkiem jest zachodzi quasi symetria to...

test z wykladu w zadaniu drugim, ostatni podrozdzial kilka linijek

przeczytanie przykladu do wykladu i powtorzenie analogicznych testow dla naszego  przykladu

nie bedzie trudne i pracochlonne

# Lista 12, 13, 14

modelujemy logarytm (slajd 14)

lambda - wyraz wolny, wspolczenniki zwiazane z pojedynczymi zmiennymi

## Zadanie 1

Rozszyfrowac co to za model, roszszyfrowac i zapisac rownanie + interpretacja (slajd 26-) 

* Zmienna 1 -- SEN
* Zmienna 2 -- BIEGANIE
* Zmienna 3 -- PIES


### a. 
Mamy wyraz wolny, efekt gl odp 1 zmiennej, efekt gl odp zmiennej 3, nie bierzemy interakcji iinnych emiznnych, interpretacja = zakladamy ze 3 zmienne losowe sa niezlezne, dodatkowo ta dwojka ma rozklad rownomierny. 

ZMIENNA LOSOWA 2 BIEGANIE ma rozklad rownomierny a 1 i 3 dowolne zmienne losowe o roznych rozkladach.



a-f dopasowac do danych 

(kroki ze skryptu)

wczytac dane ankieta (zerznac kod z tablicy)

zdefiniowac model

odpalic pods. modelu

policzyc test (p-value): porownac licznosci teoretyczne i te z modelu

powtorzyc 


TABELA z p-wartosciami dla a-f
licznosci teoretyczne i licznosci z modeli a-f



In [20]:
data <- read.csv2("Ankieta.csv")
data <- mutate(data, across(c("SEN", "BIEGANIE", "PIES")))
data <- ftable(data)
data.df <- as.data.frame(data)

data.df

SEN,BIEGANIE,PIES,Freq
<fct>,<fct>,<fct>,<int>
0,0,0,6
1,0,0,5
0,1,0,1
1,1,0,5
0,0,1,2
1,0,1,5
0,1,1,2
1,1,1,14


Model ma postać:

$$
\ln{E\left(Y_{i_1, i_2, i_3}\right)} = \lambda + \lambda_{i_{1}}^1 + \lambda_{i_{3}}^3
$$

In [28]:
model <- glm(Freq ~ SEN + PIES, 
            data = data.df, family = poisson)

a <- fitted(model)

summary(model)


Call:
glm(formula = Freq ~ SEN + PIES, family = poisson, data = data.df)

Deviance Residuals: 
      1        2        3        4        5        6        7        8  
 1.9968  -0.4843  -0.9883  -0.4843  -0.7015  -1.2497  -0.7015   1.7853  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)   
(Intercept)   0.8491     0.3532   2.404  0.01621 * 
SEN1          0.9694     0.3541   2.738  0.00619 **
PIES1         0.3023     0.3198   0.945  0.34462   
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 20.468  on 7  degrees of freedom
Residual deviance: 11.166  on 5  degrees of freedom
AIC: 42.982

Number of Fisher Scoring iterations: 5


Przeprowadźmy test, czy dane pochodzą z rozważanego modelu. Hipoteza zerowa $H_0$ mówi, że dane są z modelu $[1\, 2\, 3]$, a hipoteza alternatywna $H_a$ stanowi, że dane są z modelu pełnego.

In [44]:
p_a <- 1-pchisq(deviance(model), df = df.residual(model))

p_a %>% round(4)

[1] 0.0779

Otrzymaliśmy p-wartość równą $0.0779 > 0.05$.

### b.

Model ma postać:

$$ 
\ln{E\left(Y_{i_1, i_2, i_3}\right)} = \lambda + \lambda_{i_{1}}^{(1)} + \lambda_{i_{3}}^{(3)} + \lambda_{i_{1}i_{3}}^{(13)}
$$

In [42]:
model <- glm(Freq ~ SEN + PIES + SEN*PIES, 
            data = data.df, family = poisson)

b <- fitted(model)

summary(model)


Call:
glm(formula = Freq ~ SEN + PIES + SEN * PIES, family = poisson, 
    data = data.df)

Deviance Residuals: 
     1       2       3       4       5       6       7       8  
 1.212   0.000  -1.579   0.000   0.000  -1.607   0.000   1.363  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)   1.2528     0.3780   3.314 0.000918 ***
SEN1          0.3567     0.4928   0.724 0.469210    
PIES1        -0.5596     0.6268  -0.893 0.371944    
SEN1:PIES1    1.2015     0.7386   1.627 0.103791    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 20.4680  on 7  degrees of freedom
Residual deviance:  8.4013  on 4  degrees of freedom
AIC: 42.217

Number of Fisher Scoring iterations: 5


In [45]:
p_b <- 1-pchisq(deviance(model), df = df.residual(model))

p_b %>% round(4)

[1] 0.0779

Wniosek ...

### c.

...

Model ma postać:

$$
\ln{E\left(Y_{i_1, i_2, i_3}\right)} = \lambda + \lambda_{i_{1}}^{(1)} + \lambda_{i_{2}}^{(2)}+  \lambda_{i_{3}}^{(3)}
$$

In [47]:
Model ma postać:$$
\ln{E\left(Y_{i_1, i_2, i_3}\right)} = \lambda + \lambda_{i_{1}}^{(1)} + \lambda_{i_{2}}^{(2)}+  \lambda_{i_{3}}^{(3)}
$$model <- glm(Freq ~ SEN + BIEGANIE + PIES, 
            data = data.df, family = poisson)

c <- fitted(model)

summary(model)


Call:
glm(formula = Freq ~ SEN + BIEGANIE + PIES, family = poisson, 
    data = data.df)

Deviance Residuals: 
      1        2        3        4        5        6        7        8  
 2.1872  -0.2359  -1.1197  -0.7169  -0.5302  -0.9736  -0.8622   1.4785  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)   
(Intercept)   0.7437     0.3941   1.887  0.05912 . 
SEN1          0.9694     0.3541   2.738  0.00619 **
BIEGANIE1     0.2007     0.3178   0.631  0.52778   
PIES1         0.3023     0.3198   0.945  0.34462   
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 20.468  on 7  degrees of freedom
Residual deviance: 10.766  on 4  degrees of freedom
AIC: 44.581

Number of Fisher Scoring iterations: 5


In [48]:
p_c <- 1-pchisq(deviance(model), df = df.residual(model))

p_c %>% round(4)

[1] 0.0293

Wniosek ...

### d.

...

Model ma postać:

$$
\ln{E\left(Y_{i_1, i_2, i_3}\right)} = \lambda + \lambda_{i_{1}}^{(1)} + \lambda_{i_{2}}^{(2)}+  \lambda_{i_{3}}^{(3)} 
+ \lambda_{i_{1}i_{2}}^{(12)}
$$

In [88]:
Model ma postać:$$
\ln{E\left(Y_{i_1, i_2, i_3}\right)} = \lambda + \lambda_{i_{1}}^{(1)} + \lambda_{i_{2}}^{(2)}+  \lambda_{i_{3}}^{(3)} 
+ \lambda_{i_{1}i_{2}}^{(12)}
$$model <- glm(Freq ~ SEN + BIEGANIE + PIES + SEN*BIEGANIE, 
            data = data.df, family = poisson)

d <- fitted(model)

summary(model)


Call:
glm(formula = Freq ~ SEN + BIEGANIE + PIES + SEN * BIEGANIE, 
    family = poisson, data = data.df)

Deviance Residuals: 
      1        2        3        4        5        6        7        8  
 1.2711   0.3538  -0.2532  -1.1647  -1.3669  -0.3200   0.2042   0.8911  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)   
(Intercept)      1.2238     0.3985   3.071  0.00214 **
SEN1             0.2231     0.4743   0.470  0.63805   
BIEGANIE1       -0.9808     0.6770  -1.449  0.14740   
PIES1            0.3023     0.3198   0.945  0.34462   
SEN1:BIEGANIE1   1.6227     0.7816   2.076  0.03789 * 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 20.4680  on 7  degrees of freedom
Residual deviance:  5.9683  on 3  degrees of freedom
AIC: 41.784

Number of Fisher Scoring iterations: 5


In [56]:
p_d <- 1-pchisq(deviance(model), df = df.residual(model))

p_d %>% round(4)

[1] 0.1132

Wniosek ...

### e.

...

Model ma postać:

$$
\ln{E\left(Y_{i_1, i_2, i_3}\right)} = \lambda + \lambda_{i_{1}}^{(1)} + \lambda_{i_{2}}^{(2)}+  \lambda_{i_{3}}^{(3)} 
+ \lambda_{i_{1}i_{2}}^{(12)} + \lambda_{i_{1}i_{3}}^{(13)}
$$

In [49]:
Model ma postać:$$
\ln{E\left(Y_{i_1, i_2, i_3}\right)} = \lambda + \lambda_{i_{1}}^{(1)} + \lambda_{i_{2}}^{(2)}+  \lambda_{i_{3}}^{(3)} 
+ \lambda_{i_{1}i_{2}}^{(12)} + \lambda_{i_{1}i_{3}}^{(13)}
$$model <- glm(Freq ~ SEN + BIEGANIE + PIES + SEN*BIEGANIE + SEN*PIES, 
            data = data.df, family = poisson)

e <- fitted(model)

summary(model)


Call:
glm(formula = Freq ~ SEN + BIEGANIE + PIES + SEN * BIEGANIE + 
    SEN * PIES, family = poisson, data = data.df)

Deviance Residuals: 
      1        2        3        4        5        6        7        8  
 0.3917   0.7824  -0.7245  -0.6329  -0.5652  -0.6329   0.7787   0.4311  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)      1.6275     0.4207   3.869 0.000109 ***
SEN1            -0.3896     0.5852  -0.666 0.505590    
BIEGANIE1       -0.9808     0.6770  -1.449 0.147385    
PIES1           -0.5596     0.6268  -0.893 0.371937    
SEN1:BIEGANIE1   1.6227     0.7816   2.076 0.037890 *  
SEN1:PIES1       1.2015     0.7386   1.627 0.103787    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 20.4680  on 7  degrees of freedom
Residual deviance:  3.2033  on 2  degrees of freedom
AIC: 41.019

Number of Fisher Scoring iterations: 4


In [50]:
p_e <- 1-pchisq(deviance(model), df = df.residual(model))

p_e %>% round(4)

[1] 0.2016

Wniosek ...

### f.

...

Model ma postać:

$$
\ln{E\left(Y_{i_1, i_2, i_3}\right)} = \lambda + \lambda_{i_{1}}^{(1)} + \lambda_{i_{2}}^{(2)}+  \lambda_{i_{3}}^{(3)}  + \lambda_{i_{2}i_{3}}^{(23)}
$$

In [53]:
model <- glm(Freq ~ SEN + BIEGANIE + PIES + BIEGANIE*PIES, 
            data = data.df, family = poisson)

f <- fitted(model)

summary(model)


Call:
glm(formula = Freq ~ SEN + BIEGANIE + PIES + BIEGANIE * PIES, 
    family = poisson, data = data.df)

Deviance Residuals: 
       1         2         3         4         5         6         7         8  
 1.50605  -1.13193  -0.54631   0.30434   0.05371  -0.03337  -1.28303   0.68225  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)   
(Intercept)       1.1069     0.3960   2.795  0.00519 **
SEN1              0.9694     0.3541   2.738  0.00619 **
BIEGANIE1        -0.6061     0.5075  -1.194  0.23236   
PIES1            -0.4520     0.4835  -0.935  0.34988   
BIEGANIE1:PIES1   1.4328     0.6804   2.106  0.03522 * 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 20.4680  on 7  degrees of freedom
Residual deviance:  6.0561  on 3  degrees of freedom
AIC: 41.872

Number of Fisher Scoring iterations: 5


In [65]:
p_f <- 1-pchisq(deviance(model), df = df.residual(model))

p_f %>% round(4)

[1] 0.1132

#### Tabela porównawcza

...

In [91]:
ma <- matrix(c(data.df$Freq, a, b, c, d, e, f), byrow=F, nrow=8)


colnames(ma) <- c("teoretyczne", "[1 3]", "[13]", "[1 2 3]", "[12 3]", "[12 13]", "[1 23]")

ma

teoretyczne,[1 3],[13],[1 2 3],[12 3],[12 13],[1 23]
6,2.3375,3.025,2.10375,3.400,5.090909,3.025
5,6.1625,7.975,5.54625,4.250,3.448276,7.975
1,2.3375,1.650,2.57125,1.275,1.909091,1.650
5,6.1625,4.350,6.77875,8.075,6.551724,4.350
2,3.1625,1.925,2.84625,4.600,2.909091,1.925
5,8.3375,5.075,7.50375,5.750,6.551724,5.075
2,3.1625,4.400,3.47875,1.725,1.090909,4.400
14,8.3375,11.600,9.17125,10.925,12.448276,11.600


In [78]:
# HIDE

m <- matrix(c(p_a, p_b, p_c, p_d, p_e, p_f), byrow=F, nrow=1)

rownames(m) <- c("p-wartość")
colnames(m) <- c("[1 3]", "[13]", "[1 2 3]", "[12 3]", "[12 13]", "[1 23]")

m %>% round(4)

,[1 3],[13],[1 2 3],[12 3],[12 13],[1 23]
p-wartość,0.0779,0.0779,0.0293,0.1132,0.2016,0.1132


In [72]:
?dimnames

In [9]:
mod1 <- glm(Freq ~ cigarette + marijuana + alcohol, 
            data = seniors.df, family = poisson)

summary(mod1)

#h0 ze dane z modelu 1 2 3
#ha dane pochdoza z pelnego modelu


# wyznaczamy p-wartość
1-pchisq(deviance(mod1), df = df.residual(mod1))

# p-w = 0 => odrzucamy h0, bierzemy z pelnego modelu


Call:
glm(formula = Freq ~ cigarette + marijuana + alcohol, family = poisson, 
    data = seniors.df)

Deviance Residuals: 
      1        2        3        4        5        6        7        8  
 14.522  -17.683   -7.817    3.426  -12.440   -8.832   -8.436   19.639  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   4.17254    0.06496  64.234  < 2e-16 ***
cigaretteyes  0.64931    0.04415  14.707  < 2e-16 ***
marijuanayes -0.31542    0.04244  -7.431 1.08e-13 ***
alcoholyes    1.78511    0.05976  29.872  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 2851.5  on 7  degrees of freedom
Residual deviance: 1286.0  on 4  degrees of freedom
AIC: 1343.1

Number of Fisher Scoring iterations: 6


[1] 0

In [11]:
exp(4.17254+0.64931-0.31542+1.78511)

[1] 539.9843

In [1]:
### ANALIZOWANE DANE POCHODZĄ Z KSIĄŻKI Agresti (Tabela 6.3)

## PRZYGOTOWUJEMY DANE DO ANALIZY

# dane
seniors <- array(data = c(911, 44, 538, 456, 3, 2, 43, 279), 
                 dim = c(2,2,2), 
                 dimnames = list("cigarette" = c("yes","no"),
                                 "marijuana" = c("yes","no"),
                                 "alcohol" = c("yes","no")))
seniors

# dane - przedstawienie w tablicy
ftable(seniors, row.vars = c("alcohol","cigarette"))

# dodajmy liczności brzegowe
addmargins(seniors)

# spójrzmy na proporcje
prop.table(seniors, margin = c(1,3))

# zmieniamy format danych
seniors.df <- as.data.frame(as.table(seniors))
seniors.df[,-4] <- lapply(seniors.df[,-4], relevel, ref = "no") #ustawiamy referencje na "no"
seniors.df

## MODELOWANIE, Model 1


mod1 <- glm(Freq ~ cigarette + marijuana + alcohol, 
            data = seniors.df, family = poisson)

summary(mod1)

# wyznaczamy p-wartość
1-pchisq(deviance(mod1), df = df.residual(mod1))

# porównujemy liczności z modelem
cbind(mod1$data, fitted(mod1))
#cbind(mod2$data, fitted(mod2))
exp(4.17254+0.64931-0.31542+1.78511) #wartość dla cigaretteyes, marijuanayes, alcoholyes
exp(4.17254+0-0.31542+1.78511) #wartość dla cigaretteno, marijuanayes, alcoholyes itd.

# trzeba dac exponente bo modelujemy logarytm

## MODELOWANIE, Model 2

mod2 <- glm(Freq ~ cigarette + marijuana + alcohol + 
              cigarette*marijuana + marijuana*alcohol + cigarette*alcohol, 
            data = seniors.df, family = poisson)
summary(mod2)

# model => relacja miedzy zmiennymi jest stala niezaleznie od 3 zmiennej bo nie ma xyz


# równoważnie
#mod2 <- glm(Freq ~ (cigarette + marijuana + alcohol)^2, (do pot 2 => wez wszystkie interakcje 2 rzedu)
#             data = seniors.df, family = poisson)
#lub (hierarchicznie uporzadkowany , relacje niezerowe -> wsp niezerowe rowniez)
#mod2 <- glm(Freq ~ 
#              cigarette*marijuana + marijuana*alcohol + cigarette*alcohol, 
#            data = seniors.df, family = poisson)

#wyznaczamy p-wartość
1-pchisq(deviance(mod2), df = df.residual(mod2))

# porównujemy liczności z modelem
cbind(mod2$data, fitted(mod2))

exp(5.63342-1.88667-5.30904+0.48772+2.84789+2.05453+2.98601) #wartość dla cigaretteyes, marijuanayes, alcoholyes
exp(5.63342+0-5.30904+0.48772+0+0+2.98601) #wartość dla cigaretteno, marijuanayes, alcoholyes itd.

#test <- anova(mod1,mod2)
#1-pchisq(test$Deviance[2], df = test$Df[2])

, , alcohol = yes

         marijuana
cigarette yes  no
      yes 911 538
      no   44 456

, , alcohol = no

         marijuana
cigarette yes  no
      yes   3  43
      no    2 279

                  marijuana yes  no
alcohol cigarette                  
yes     yes                 911 538
        no                   44 456
no      yes                   3  43
        no                    2 279

, , alcohol = yes

         marijuana
cigarette yes  no  Sum
      yes 911 538 1449
      no   44 456  500
      Sum 955 994 1949

, , alcohol = no

         marijuana
cigarette yes  no Sum
      yes   3  43  46
      no    2 279 281
      Sum   5 322 327

, , alcohol = Sum

         marijuana
cigarette yes   no  Sum
      yes 914  581 1495
      no   46  735  781
      Sum 960 1316 2276

, , alcohol = yes

         marijuana
cigarette       yes        no
      yes 0.6287095 0.3712905
      no  0.0880000 0.9120000

, , alcohol = no

         marijuana
cigarette         yes        no
      yes 0.065217391 0.9347826
      no  0.007117438 0.9928826

cigarette,marijuana,alcohol,Freq
<fct>,<fct>,<fct>,<dbl>
yes,yes,yes,911
no,yes,yes,44
yes,no,yes,538
no,no,yes,456
yes,yes,no,3
no,yes,no,2
yes,no,no,43
no,no,no,279



Call:
glm(formula = Freq ~ cigarette + marijuana + alcohol, family = poisson, 
    data = seniors.df)

Deviance Residuals: 
      1        2        3        4        5        6        7        8  
 14.522  -17.683   -7.817    3.426  -12.440   -8.832   -8.436   19.639  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   4.17254    0.06496  64.234  < 2e-16 ***
cigaretteyes  0.64931    0.04415  14.707  < 2e-16 ***
marijuanayes -0.31542    0.04244  -7.431 1.08e-13 ***
alcoholyes    1.78511    0.05976  29.872  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 2851.5  on 7  degrees of freedom
Residual deviance: 1286.0  on 4  degrees of freedom
AIC: 1343.1

Number of Fisher Scoring iterations: 6


[1] 0

,cigarette,marijuana,alcohol,Freq,fitted(mod1)
,<fct>,<fct>,<fct>,<dbl>,<dbl>
1,yes,yes,yes,911,539.98258
2,no,yes,yes,44,282.09123
3,yes,no,yes,538,740.22612
4,no,no,yes,456,386.70007
5,yes,yes,no,3,90.59739
6,no,yes,no,2,47.32880
7,yes,no,no,43,124.19392
8,no,no,no,279,64.87990


ERROR: Error in cbind(mod2$data, fitted(mod2)): object 'mod2' not found


In [12]:
mod2 <- glm(Freq ~ cigarette + marijuana + alcohol + 
              cigarette*marijuana + marijuana*alcohol + cigarette*alcohol, 
            data = seniors.df, family = poisson)
summary(mod2)

# równoważnie
#mod2 <- glm(Freq ~ (cigarette + marijuana + alcohol)^2, 
#             data = seniors.df, family = poisson)
#lub
#mod2 <- glm(Freq ~ 
#              cigarette*marijuana + marijuana*alcohol + cigarette*alcohol, 
#            data = seniors.df, family = poisson)

#wyznaczamy p-wartość
1-pchisq(deviance(mod2), df = df.residual(mod2))

# porównujemy liczności z modelem
cbind(mod2$data, fitted(mod2))

exp(5.63342-1.88667-5.30904+0.48772+2.84789+2.05453+2.98601) #wartość dla cigaretteyes, marijuanayes, alcoholyes
exp(5.63342+0-5.30904+0.48772+0+0+2.98601) #wartość dla cigaretteno, marijuanayes, alcoholyes itd.



Call:
glm(formula = Freq ~ cigarette + marijuana + alcohol + cigarette * 
    marijuana + marijuana * alcohol + cigarette * alcohol, family = poisson, 
    data = seniors.df)

Deviance Residuals: 
       1         2         3         4         5         6         7         8  
 0.02044  -0.09256  -0.02658   0.02890  -0.33428   0.49134   0.09452  -0.03690  

Coefficients:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)                5.63342    0.05970  94.361  < 2e-16 ***
cigaretteyes              -1.88667    0.16270 -11.596  < 2e-16 ***
marijuanayes              -5.30904    0.47520 -11.172  < 2e-16 ***
alcoholyes                 0.48772    0.07577   6.437 1.22e-10 ***
cigaretteyes:marijuanayes  2.84789    0.16384  17.382  < 2e-16 ***
marijuanayes:alcoholyes    2.98601    0.46468   6.426 1.31e-10 ***
cigaretteyes:alcoholyes    2.05453    0.17406  11.803  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter

[1] 0.5408396

,cigarette,marijuana,alcohol,Freq,fitted(mod2)
,<fct>,<fct>,<fct>,<dbl>,<dbl>
1,yes,yes,yes,911,910.38317
2,no,yes,yes,44,44.61683
3,yes,no,yes,538,538.61683
4,no,no,yes,456,455.38317
5,yes,yes,no,3,3.61683
6,no,yes,no,2,1.38317
7,yes,no,no,43,42.38317
8,no,no,no,279,279.61683


[1] 910.3781

[1] 44.61678